In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# PCA - Conceitos Básicos
Hoje vamos conhecer melhor essa ferramenta famosa e poderosa. Para iniciar, vamos começar com a lógica da sua implementação. Ele tenta reduzir os dados de forma que a nova representação possua a mesma variância. Explicando assim é difícil de entender, então vamos ao código!

In [2]:
# A primeira parte é entender o que é variância. Variância é uma medida de 
# disperção dos dados. Enquanto que a média e mediana indicam onde os dados
# se encontram, a variância e o desvio padrão indicam a sua disperção.

# Vamos para exemplos:

array_1 = np.array([5,5.5,4.5,6,4,5])
array_2 = np.array([0,0,0,10,10,10])

# Vamos calcular os indicadores citados anteriormente
mean_1, median_1, var_1, std_1 = np.mean(array_1), np.median(array_1), np.var(array_1), np.std(array_1)
mean_2, median_2, var_2, std_2 = np.mean(array_2), np.median(array_2), np.var(array_2), np.std(array_2)

# Mostra os resultados
print(mean_1, mean_2)
print(median_1, median_2)
print(var_1, var_2)
print(std_1, std_2)

5.0 5.0
5.0 5.0
0.4166666666666667 25.0
0.6454972243679028 5.0


Podemos ver que, embora essas duas listas possuam médias e medianas iguais, seu desvio padrão (std) e sua variância são totalmente diferentes. Isso faz sentido, visto que a primeira lista possui valores próximos, enquanto que a segunda possui valores bem distantes.

Agora que sabemos o que é a variância de um conjunto de dados, vamos entender o que é a variância entre duas variáveis.

In [3]:
# Covariância (variância entre duas variáveis) é uma medida que mostra
# a dependência de uma variável com outra. Covariância igual a 0 indica
# que as variáveis são DESCORRELACIONADAS (não confundir com independente,
# explico depois).

def cov(data_1, data_2):
    """
    Função que calcula covariância entre dois valores. 

    Parameters
    ----------
    data_1 : numpy.array
        Conjunto de dados.
    data_2 : numpy.array
        Segundo connjunto de dados.


    Returns
    -------
    covariânce : float
        Valor da covariância.
    """

    mean_1, mean_2 = data_1.mean(), data_2.mean() # calculando a média
    n = len(data_1)
    return ((data_1 - mean_1) * (data_2 - mean_2)).sum()/(n-1)

print(cov(array_1, array_2))

0.0


Nesses nossos dados, temos que nossos dados não possuem dependência direta. Ou seja, são descorrelacionadas. Mas por que eu não posso inferir que elas são independentes? 

Existe uma diferença entre ser descorrelacionada e independente. Duas variáveis X e Y são descorrelacionadas quando a Cov(X,Y) é igual a 0 e são independentes quando a Cov( f(X), g(Y) ) é igual a 0 para qualquer função f e g. Será que nossos pontos são independentes?

In [4]:
# Faremos 3 testes com eles

print(cov(array_1, array_2**3))               # Aqui, a cov continua zero! 
print(cov(np.log(array_1), array_2))          # Aqui, a cov é -0.03....
print(cov(np.exp(array_1), np.exp(array_2)))  # Aqui, a cov é 2.71e5....

# Logo, eles não são independentes

0.0
-0.03077165866675391
271613.37469721073


Para o próximo passo, vamos falar sobre a matriz de covariância. Ela é o centro do algoritmo de PCA, então é de extrema importância falar dela.

In [5]:
# Essa matriz é uma matriz quadrada que mostra todas as covariâncias entre N  
# variáveis aleatórias. Na sua diagonal principal, há a covariância de cada
# variável com ela mesma. Vamos fazer um experimento com os nossos pontos.

# Covariância entre o array_1 com ele mesmo.
cov_11 = cov(array_1, array_1)

# Covariância entre o array_2 com ele mesmo.
cov_22 = cov(array_2, array_2)

# Covariância entre o array_1 e o array_2.
cov_12 = cov(array_1, array_2)

print(cov_11, cov_22, cov_12)

# Vamos calcular a matriz de covariância dessas duas variáveis usando a função
# do numpy: np.cov

matriz = np.cov(array_1, array_2)
print(matriz)

0.5 30.0 0.0
[[ 0.5  0. ]
 [ 0.  30. ]]


Temos então uma matriz 2x2 por estarmos usando duas variáveis aleatórias. O elemento Aij (lembre-se, i -> linha, j -> coluna) da matriz é o valor da covariância entre a variável i com a variável j. No nosso exemplo, A11 é igual a covariância da variável 1 com a 1, A12 = A21 é a covariância entre 1 com 2 e A22 é a covariância entre 2 e 2.

Agora, por que essa matriz é importante pro PCA? Porque o seu objetivo é criar variáveis descorrelacionadas. Vamos entender mais de perto.

In [6]:
# Vamos criar um conjunto de dados aleatórios, ver sua matriz de covariância,
# aplicar o PCA do Sklearn e calcular a matriz de covariância nos novos dados.

np.random.seed(100) 

dados = np.random.random((100,3))*10 # Cria 100 pontos de 3 dimensões

print("Cov dos dados originais:")
print(np.cov(dados.T)) # O .T é para ele entender que possui 3 dimensões, não 100

from sklearn.decomposition import PCA # PCA do Sklearn

dados_pca = PCA().fit_transform(dados) # Transforma nossos dados

print("\n\nCov dos dados pós pca:")
print(np.cov(dados_pca.T))

Cov dos dados originais:
[[ 8.224208    0.43011632 -1.58685266]
 [ 0.43011632  8.62601916  0.30802666]
 [-1.58685266  0.30802666  8.41397601]]


Cov dos dados pós pca:
[[ 9.91294324e+00 -1.69262803e-16 -1.65954591e-15]
 [-1.69262803e-16  8.75723187e+00  1.97308034e-15]
 [-1.65954591e-15  1.97308034e-15  6.59402806e+00]]


Vamos entender o resultado. Inicialmente, nossos dados não eram descorrelacionados, pois os valores fora da diagonal principal na matriz de correlação eram diferente de 0. Depois de aplicar o PCA, os valores fora da diagonal principal são muito próximos de 0. Então o PCA está transformando nossos dados em dados descorrelacionados.

Por que criar dados descorrelacionados? Pois assim a covariância fica somente na diagonal principal da matriz, e conseguimos ver quais variáveis possuem mais covariância e qual possui menos. 

# PCA - Código
Então o objetivo do PCA é criar um novo conjunto de dados descorrelacionado a partir de um conjunto de dados existente. Em outras palavras, ele quer transformar uma matriz em uma matriz com valores apenas na diagonal, ou seja, ele quer **diagonalizar** a matriz. Esse problema é conhecido em algebra linear, e sua explicação está nesse [link](https://pt.wikipedia.org/wiki/Matriz_diagonaliz%C3%A1vel). 

Vamos codar!

In [7]:
# Para diagonalizar uma matriz, precisamos calcular os autovetores e  
# os autovalores dela. Utilizaremos a funçao do numpy: numpy.linalg.eig.

cova_matriz = np.cov(dados.T)

val, vec = np.linalg.eig(cova_matriz)
print("Os autovalores são:")
print(val)
print("Os respectivos autovetores são:")
print(vec)

Os autovalores são:
[6.59402806 9.91294324 8.75723187]
Os respectivos autovetores são:
[[-0.70843442  0.69150895  0.14119506]
 [ 0.25000645  0.05879045  0.96645769]
 [-0.66001322 -0.71997157  0.21453088]]


Temos nossos autovalores (que indicam a covariância dos novos **componentes**) e nossos autovetores (que são nossos novos **componentes**). Componentes são os novos eixos que vamos criar com o PCA.

**Isso é de extrema importância**: o PCA transforma os seus dados. O primeiro eixo dos dados originais não tem o mesmo significado que o primeiro componente do PCA, você perde **interpretabilidade** dos dados quando aplica PCA.

Vamos continuar com o algoritmo.

In [8]:
# Agora que temos os autovetores, vamos tentar criar os dados que o PCA do 
# Sklearn criou. Primeiramente, ordenamos os autovetores baseado no  seu 
# respectivo autovalor.


args = np.argsort(-val) # Essa função retorna os indices do menor para o maior,
                        # como queremos o inverso, multiplicamos por -1.

val_sort = val[args]   # Ordena os autovalores
vec_sort = vec[:,args] # Ordena os autovetores


dados_mean = dados - dados.mean(axis=0) # Vamos centralizar os dados

dados_novos = dados_mean.dot(vec_sort) # Vamos calcular nossos novos componentes

print("Nossos dados:")
print(dados_novos[:3,:])

print("\n\nDados PCA Sklearn:")
print(dados_pca[:3,:])

Nossos dados:
[[ 0.09252165 -2.22066347 -0.24436895]
 [ 4.19679286 -5.08977681 -1.06403023]
 [ 3.36714311  2.63289181  2.12181569]]


Dados PCA Sklearn:
[[-0.09252165  2.22066347  0.24436895]
 [-4.19679286  5.08977681  1.06403023]
 [-3.36714311 -2.63289181 -2.12181569]]


Podemos ver que há diferença entre o sinal, por causa que o PCA do Sklearn utiliza outra forma (SVD) para encontrar os componentes. Isso não influenciar o resultado, visto que **todos** os pontos só possuem o sinal diferente, o seu valor é o mesmo (visualmente, eles só estariam rotacionados 180º, a informação aprendida é a mesma).

Mas ainda não houve a redução, só mudança de base. Vamos para a parte final do algoritmo.

In [9]:
# Para reduzir a dimensionalidade, temos que definir quanto de informação
# queremos manter. O que isso significa? Cada componente possui informação
# do problema, e queremos manter aqueles que possuem mais informação.
# A informação, no algoritmo do PCA, é a covariância dos dados, pois
# ele entende que variaveis com pouca variância trazem pouca informação, 
# mas aquelas que variam muito são importantes. 
#
# Visto isso, a informação está relacionada com os autovalores

print("Nossos autovalores:")
print(val_sort)

info_total = val_sort.sum() # Aqui calculamos a informação total

print(f"\n\nInformação total: {info_total}") 

print("\n\nInformação de cada autovalor:")
print(val_sort/info_total)

print("\n\nInformação pelo Sklearn:")
print(PCA().fit(dados).explained_variance_ratio_) # É igual ao nosso

Nossos autovalores:
[9.91294324 8.75723187 6.59402806]


Informação total: 25.26420316222461


Informação de cada autovalor:
[0.3923711  0.34662609 0.26100281]


Informação pelo Sklearn:
[0.3923711  0.34662609 0.26100281]


A ultima parte do algoritmo é decidirmos o quanto de informação queremos. Por exemplo, se quisessemos manter 70% da informação no nosso problema, o ultimo componente seria excluido, pois os 2 primeiros já garantem 70%. Valores normais na literatura são 95% e 99%.

Vamos fazer o mesmo processo com uma reta.

In [11]:
np.random.seed(100)

x = np.random.random((100,1))
y = 2*x + 0.1*np.random.random((100,1))

dados_reta = np.concatenate([x,y], axis=1)
cova_reta = np.cov(dados_reta.T)
val_reta, vec_reta = np.linalg.eig(cova_reta)

args_reta = np.argsort(-val_reta) 

val_reta_sort = val_reta[args_reta]   
vec_reta_sort = vec_reta[:,args_reta] 

dados_reta_mean = dados_reta - dados_reta.mean(axis=0) 
dados_novos_reta = dados_reta_mean.dot(vec_reta_sort) 

info_total_reta = val_reta_sort.sum()
print("Informação de cada autovalor:")
print(val_reta_sort/info_total_reta)

Informação de cada autovalor:
[9.99604608e-01 3.95391716e-04]


Podemos ver que, para uma reta de 2 dimensões, o PCA consegue criar um componente com 99.6% de informação. Isso se dá pelo fato de que o PCA é um algoritmo **linear**! Mas isso não significa que ele seja ruim com dados não-lineares, pelo contrário, é muito comum utilizar ele nesses casos pois ele ainda é capaz de reduzir dimensão (mas não na mesma magnitude de um algoritmo não linear).

Vamos colocar tudo junto e fazer uma função?

In [0]:
def PCATuring(dados, nova_dimensao):
    """
    Função de PCA.

    Parameters
    ----------
    dados : np.array
        Dados para aplicação do PCA. Dimensão NxD, onde N é 
        o número de pontos e D a dimensão dos dados.
    nova_dimensao : int
        Valor para nova dimensão.

    Returns
    -------
    novos_dados : np.array
        Dados criados pelo PCA.
    info : np.array
        Array com a informação de cada Componente.

    """
    cova = np.cov(dados.T)
    val, vec = np.linalg.eig(cova)

    args = np.argsort(-val) 

    val_sort = val[args]   
    vec_sort = vec[:,args] 

    dados_mean = dados - dados.mean(axis=0) 
    dados_novos = dados_mean.dot(vec_sort) 

    info = val_sort/val_sort.sum()
    return dados_novos[:,:nova_dimensao], info

Pronto! Acabamos o algoritmo. 

# Comentários adicionais
O PCA, como foi dito anteriormente, é um algoritmo linear. Mesmo assim, ele é aplicado em diversos problemas, desde dados mais simples até sinais biológicos, como EEG. Essa sua fama é explicada por esse algoritmo não precisar de nenhum parâmetro, ele só depende dos dados do problema.

Cabe ressaltar que, embora ele seja aplicável sempre, há alguns detalhes que devem ser considerados antes de aplicar. O primeiro é a **escala**. Não faz sentido comparar variância de uma variável de média 0.01 e std 0.005 com uma variável de média 100000 e std de 10000. Nesses casos, é bom **padronizar** os dados, ou seja, subtrair a média e dividir pelo desvio padrão. Isso torna a comparação mais justa. Um detalhe é que, quando se padroniza os dados, a matriz de covariância se torna a matriz de **correlação**.